In [1]:
import os
os.chdir('../../..')
print(os.getcwd()) # Ensure this is you Amphion root path, otherwise change the above path to you amphion root path
assert os.path.isfile('./README.md') # make sure the current path is Amphion root path
import sys
sys.path.append('.')

/home/chris/repo/Amphion


In [2]:
# put your cheackpoint file (.bin) in the root path of AmphionVALLEv2
# or use your own pretrained weights
ar_model_path = 'ckpts/valle_ar_mls_196000.bin'  # huggingface-cli download amphion/valle valle_ar_mls_196000.bin valle_nar_mls_164000.bin --local-dir ckpts
nar_model_path = 'ckpts/valle_nar_mls_164000.bin'
speechtokenizer_path = 'ckpts/speechtokenizer_hubert_avg' # huggingface-cli download amphion/valle speechtokenizer_hubert_avg/SpeechTokenizer.pt speechtokenizer_hubert_avg/config.json --local-dir ckpts

In [3]:
device = 'cuda' # change to 'cuda' if you have gpu

In [5]:
!pip install easydict

In [6]:
from models.tts.valle_v2.valle_inference import ValleInference
# change to device='cuda' to use CUDA GPU for fast inference
# change "use_vocos" to True would give better sound quality
# If you meet problem with network, you could set "use_vocos=False", though would give bad quality
model = ValleInference(ar_path=ar_model_path, nar_path=nar_model_path, speechtokenizer_path=speechtokenizer_path, device=device)
# model = ValleInference(use_vocos=False, ar_path=ar_model_path, nar_path=nar_model_path, device='cuda')

/home/chris/.local/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded SpeechTokenizer from ckpts/speechtokenizer_hubert_avg/config.json and ckpts/speechtokenizer_hubert_avg/SpeechTokenizer.pt


In [25]:
# prepare inference data
import librosa
import torch
wav, _ = librosa.load('./egs/tts/VALLE_V2/example.wav', sr=16000)
wav = torch.tensor(wav, dtype=torch.float32)
from IPython.display import Audio
Audio(wav, rate = 16000)

In [ ]:
!pip install g2p_en


In [30]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/chris/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [53]:
# The transcript of the prompt part
prompt_transcript_text = 'and keeping eternity before the eyes'

# Here are the words you want the model to output
target_transcript_text = 'I can say whatever the fuck I want to say'
from models.tts.valle_v2.g2p_processor import G2pProcessor
g2p = G2pProcessor()
prompt_transcript = g2p(prompt_transcript_text, 'en')[1]
target_transcript = g2p(target_transcript_text, 'en')[1]

In [54]:
prompt_transcript = torch.tensor(prompt_transcript).long()
target_transcript = torch.tensor(target_transcript).long()
transcript = torch.cat([prompt_transcript, target_transcript], dim=-1)
batch = {
        'speech': wav.unsqueeze(0),
        'phone_ids': transcript.unsqueeze(0),
}

In [55]:
# print the contents of the model input
# `phone_ids` contains a concatenation of `prompt_transcript` and `target_transcript` 
batch

{'speech': tensor([[ 3.0518e-05,  3.0518e-05,  3.0518e-05,  ..., -3.0518e-05,
          -3.0518e-05,  3.0518e-05]]),
 'phone_ids': tensor([[  5,  28, 149,  72, 219, 134, 127, 170, 115, 147, 219, 113, 185,  91,
          149,  30, 185, 123, 219,  65, 115, 106,  43, 172, 219,  73,  29, 219,
           59, 214,   6,   5,  59, 219, 134,  24, 145, 219, 174,  99, 219, 207,
           36, 185,  82, 206,  87, 219,  73,  29, 219, 104,  33, 135, 219,  59,
          219, 207,  15, 149, 181, 219, 180, 199, 219, 174,  99,   6]])}

In [56]:
configs = [dict(
    top_p=0.9,
    top_k=5,
    temperature=0.95,
    repeat_penalty=1.0,
    max_length=2000,
    num_beams=1,
)] # model inference hyperparameters
output_wav = model(batch, configs)

In [57]:
output_wav # The output wav is a tensor of shape [1,1,T]

tensor([[[-1.1117e-05, -2.5689e-05, -4.7946e-05,  ..., -1.8422e-02,
          -1.5326e-02, -8.3061e-03]]], device='cuda:0')

In [58]:
print(f'prompt_transcript : {prompt_transcript_text}')
print(f'target_transcript : {target_transcript_text}')
# Audio(output_wav.squeeze(0), rate = 16000)
Audio(output_wav.squeeze(0).cpu().numpy(), rate=16000)


prompt_transcript : and keeping eternity before the eyes
target_transcript : I can say whatever the fuck I want to say


In [59]:
import torchaudio
# torchaudio.save('out.wav', output_wav.squeeze(0), 16000)
# torchaudio.save('out.wav', output_wav.squeeze(0).cpu(), 16000)
